In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import os

# Verify working directory contains data/ folder

print("Data folder contents:", os.listdir("data"))

# Load CSV datasets with relative paths
lap_times = pd.read_csv("data/lap_times.csv")
races = pd.read_csv("data/races.csv")
drivers = pd.read_csv("data/drivers.csv")

# Filter for 2020 Turkish GP (rainy race)
races_2020 = races[races["year"] == 2020]
try:
    turkish_gp = races_2020[races_2020["name"].str.contains("Turkish")]["raceId"].iloc[0]
except IndexError:
    print("Turkish GP not found, using first 2020 race")
    turkish_gp = races_2020["raceId"].iloc[0]
lap_times_2020 = lap_times[lap_times["raceId"] == turkish_gp]

# Simulate ADS-like overtaking decision
results = []
for driver_id in lap_times_2020["driverId"].unique():
    driver_laps = lap_times_2020[lap_times_2020["driverId"] == driver_id].sort_values("lap")
    if len(driver_laps) > 1:
        # Calculate speed (inverse of lap time) and gap to next car
        lap_times_ms = driver_laps["milliseconds"].values
        speeds = 1 / (lap_times_ms / 1000)  # Speed in arbitrary units
        gaps = np.random.uniform(0.5, 2.0, len(lap_times_ms))  # Simulated gaps (meters)
        rain_factor = 0.8  # Reduced grip in rain
        
        # ADS decision: Overtake if speed advantage > threshold
        threshold = np.mean(speeds) * 1.1 * rain_factor  # 10% faster, adjusted for rain
        for i, (lap, speed, gap) in enumerate(zip(driver_laps["lap"], speeds, gaps)):
            decision = 1 if speed > threshold and gap < 1.0 else 0  # Overtake if fast and close
            # Success: Overtaking improves next lap time
            next_lap_time = lap_times_ms[i + 1] if i + 1 < len(lap_times_ms) else lap_times_ms[-1]
            current_lap_time = lap_times_ms[i]
            success = 1 if decision == 1 and next_lap_time < current_lap_time else 0
            results.append({
                "Driver": driver_id,
                "Lap": lap,
                "Scenario": "Rain",
                "OvertakeDecision": decision,
                "OvertakeSuccess": success
            })

# Save results
pd.DataFrame(results).to_csv("results.csv", index=False)
print("Results saved to results.csv")

Data folder contents: ['circuits.csv', 'constructors.csv', 'constructor_results.csv', 'constructor_standings.csv', 'drivers.csv', 'driver_standings.csv', 'lap_times.csv', 'pit_stops.csv', 'qualifying.csv', 'races.csv', 'results.csv', 'seasons.csv', 'sprint_results.csv', 'status.csv']
Results saved to results.csv
